In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import tushare as ts
import openpyxl as oxl
import os
from collections import OrderedDict
import datetime
import json


def load_from_local():
    
    with open('__ HKEX __ HKEXnews __.html','r',encoding="utf-8") as f:
        return f.read()

def get_first_part():

    temp_html = soup.find("table",{"id":"Table5"})
    temp_html = temp_html.find_all('td',{"class":"arial12black"})

    first_dict = {}
    temp_string = temp_html[1].get_text().strip()
    temp_string = datetime.datetime.strptime(temp_string,"%d/%m/%Y")
    temp_string = temp_string.strftime("%Y-%m-%d")
    first_dict['last_date'] = temp_string
    first_dict['stock_code'] = temp_html[3].get_text().strip()
    first_dict['stock_name'] = temp_html[5].get_text().strip()

    return first_dict

def get_second_part():

    temp_html = soup.find("div",{"id":{"pnlResultSummary"}})
    temp_html = temp_html.find_all("span",{"class":"mobilezoom"})

    second_dict = {}
    second_dict['hold_volumn'] = temp_html[0].get_text().strip()
    second_dict['people_number'] = temp_html[1].get_text().strip()
    second_dict['hold_precent'] = temp_html[2].get_text().strip()
    second_dict['all_volumn'] = temp_html[6].get_text().strip()

    return second_dict

def get_third_part(first_dict):

    temp_html = soup.find("table",{"id":{"participantShareholdingList"}})
    temp_html = temp_html.find_all('tr')

    row_number = len(temp_html)
    col_number = len(temp_html[0].find_all('td'))

    d = {}
    for i in range(col_number):
        d[i] = []

    for i in range(1,row_number):
        temp_tr = temp_html[i]
        temp_td = temp_tr.find_all('td')
    
        if len(temp_td) !=col_number:
            continue
        
        for j in range(col_number):
            temp_string = temp_td[j].get_text().strip()
            d[j].append(temp_string)

    last_pd = pd.DataFrame(d)
    
    last_dict = {}
    for i in range(len(last_pd)):
        last_dict[last_pd.iat[i,0]] = last_pd.iat[i,1]

    name_dict.update(last_dict)

    new_pd = pd.DataFrame(last_pd[3])
    new_pd = new_pd.T
    new_pd.columns = last_pd[0]

    last_date = first_dict['last_date']
    time_span = pd.Timestamp(last_date)
    new_pd.index = [time_span]

    temp_string = datetime.datetime.strptime(last_date,'%Y-%m-%d')
    end_date = temp_string.date()
    start_date = end_date - datetime.timedelta(10)

    temp_df = ts.get_k_data('601888',start=start_date.isoformat(),end=end_date.isoformat())

    if temp_df.empty:
        #  close_data[last_date] = list(close_data.values())[-1]
        close_data[str(time_span)] = 0 
    else:
        close_data[str(time_span)] = temp_df.iat[-1,2]

    return new_pd

def save_to_excel(first_dict,second_dict,merge_pd):

    wb = oxl.Workbook()
    ws = wb.create_sheet(index=0,title='oxl-sheet')
    
    ws.cell(row=1,column=1).value = '持股日期'
    ws.cell(row=1,column=2).value = first_dict['last_date']
    ws.cell(row=1,column=3).value = '股票代码'
    ws.cell(row=1,column=4).value = first_dict['stock_code']
    ws.cell(row=1,column=5).value = '股票名称'
    ws.cell(row=1,column=6).value = first_dict['stock_name']

    ws.cell(row=3,column=1).value = '中央结算系统持股量'
    ws.cell(row=4,column=1).value = second_dict['hold_volumn']
    ws.cell(row=3,column=2).value = '参与者数目'
    ws.cell(row=4,column=2).value = second_dict['people_number']
    ws.cell(row=3,column=3).value = '总数百分比'
    ws.cell(row=4,column=3).value = second_dict['hold_precent']
    ws.cell(row=3,column=4).value = '全部持股量'
    ws.cell(row=4,column=4).value = second_dict['all_volumn']

    columns = merge_pd.columns
    index = merge_pd.index
    
    row_number = len(index)
    col_number = len(columns)

    ws.cell(row=8,column=1).value = '日期'
    ws.cell(row=8,column=2).value = '收盘价'

    for i in range(row_number):
        ws.cell(row=i+9,column=1).value = str(index[i])[0:10]
        ws.cell(row=i+9,column=2).value = close_data.get(str(index[i]),0)

    for i in range(col_number):
        ws.cell(row=8,column=i+3).value = columns[i]
        ws.cell(row=7,column=i+3).value = name_dict[columns[i]] 

    for i in range(row_number):
        for j in range(col_number):
            ws.cell(row=9+i,column=j+3).value = merge_pd.iat[i,j]

    wb.save('test.xlsx')
    print("数据已写入excel中!")

def plot_10(merge_pd):
    '''
    画前10大持仓股的曲线图
    '''
    temp_pd = merge_pd.sort_values(merge_pd.index[0],axis=1,ascendind=False)
    
    number = min(10,len(merge_pd))
    
    for i in range(number):
        pass


def save_to_config():

    first_dict = get_first_part()
    last_date = first_dict['last_date']

    if str(pd.Timestamp(last_date)) in close_data:
        print("该日期数据已经存在!")
        save_to_excel(config_dict['first_dict'],config_dict['second_dict'],pd.read_json(config_dict['old_pd'])) 
        return

    second_dict = get_second_part()
    third_pd = get_third_part(first_dict)
    merge_pd = third_pd.append(old_pd)
    merge_pd = merge_pd.sort_index(ascending=False)

    config_dict['name_dict'] = name_dict
    config_dict['close_data'] = close_data
    config_dict[last_date] = [first_dict,second_dict,third_pd.to_json()]
    config_dict['old_pd'] = merge_pd.to_json()
    config_dict['first_dict'] = first_dict
    config_dict['second_dict'] = second_dict
    config_dict['third_pd'] = third_pd.to_json()

    save_to_excel(first_dict,second_dict,merge_pd)
    
    with open('config.json','w',encoding='utf-8') as f:
        json.dump(config_dict,f)

    print('数据更新至%s!' % merge_pd.index[0])


In [26]:
URL = r'http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx'

In [27]:
s=requests.Session()

In [28]:
r=s.get(URL)

In [6]:
r

<Response [200]>

In [18]:
r.text

'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head><title>\r\n\t:: HKEX :: HKEXnews ::\r\n</title><link href="css/hkex_css.css" rel="stylesheet" type="text/css" />\r\n    <script type="text/javascript" src="/script/hkex_scripts.js"></script>\r\n    <script type="text/javascript" src="/script/hkex_common_c.js"></script>\r\n    <script type="text/javascript" src="/script/hkex_setting_c.js"></script>\r\n    <script type="text/javascript" src="js/search_form_var_chi.js?v=20150720"></script>\r\n    <script type="text/javascript" src="js/search_form.js?v=20150720"></script>\r\n    <script type="text/javascript">\r\n        function PopUpStockList() {\r\n            shareholdingDate = $("select[id$=\'ddlShareholdingYear\']").val() + $("select[id$=\'ddlShareholdingMonth\']").val() + $("select[id$=\'ddlShareholdingDay\']").val();\r\n            window.open(\'stockl

In [9]:
soup=BeautifulSoup(r.content,'html.parser')

In [10]:
VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
print(VIEWSTATE)

Ckzs+VaFUEDBzZtKQjL8H1DdOd0DKk5HSQ32/tzS8gH+TO1W2S8tjyZf6hmXM7avS2T/u2jdQlJ5D4T/t0hkQaQlfrd8hQKfLvJKZTYFV9m/kkOQVXsunk3uLA6HnJSrYgJAaXQdxh6mR7NrKIXPi+WK3sSA2FiV36I//ZNpgbyoOW6Cd6NncSNgKzmpxEciVxxErw==


In [11]:
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
print(VIEWSTATEGENERATOR)

3B50BBBD


In [12]:
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']
print(EVENTVALIDATION)

EDddx7fxnrdp/107CfB06ZyVcpdsQh9RV/LJjPEi4kbot5/97UrfaR99PeyxiM0tTpGLNDIJOMduS3rr1uXADZL/I2jcKp2Sj3Mw4wy1o0XeVqPB+Ph1C8fhhgd8YsPk4GZnMDLxjMhsp29QaVP7k4kgI427pwByQ4jjfdTmtMVNMDW4Ip435K0JLZTw0EauEs7LL627OdvXycUt/qQDkxj3drqss2VFubwySNJUsGfq7+nJmfikzGY2kyzMkC21ZtlCTUZBRtI/eHCl1WLrI/PPw3gKGf98ZuCRX2Wkg4FPsclYQhpCJEVKstDgaTzvDjy2RovMl2XKnR+nasAkeJUtCjx3AV4icTybEjerLsC5bsaRh+bR9UwQim4oQgqgV7Y4dOlrMrW0xCJjAmXOAEQ/V2+5ur8+kcQDLxkD3OiVCSLeasSxnqujO0cjNpe+IkKom0hI+C/j8mN5K4u/e+b5O/SI+a9yuHJfm6ONpNNPmP2aqiW3Dx0diudOvUfsRFC1dS7RTejDbrDPhCiHsIYGJF6CodoqNBjJjAZAoe8Tjsmif4eUn3c/VWKDUC9b8ZjYfjmWEn3TIyhb7Oiq8tKsZtlVhqWh716Rpi5o1uFyztFtd7lBF9DsHh1dv0kGxDbksZj3pI/hgWy2k80vpVilcqaawVmNZqR+PPA/UT97dUANMB6njkPxbwP8p2xRTyQDWZB1z9XAbgBBlPtmdp2RZUH37OWrC44938bj0dJyG3YyRQ2VsdZfGWuU9vgC4S11UlVlqh/+1woZytJmPOgUUd2La1v3fOAmgOTr1AHvZjIUesJVmQPtYpNW6wuIWPIh8ybAg1Ph560omCqiZVswSANAvcJKMEhO3hCUhcapl4Bq1Sh5kCv4p2YCtTuP9i5HRbn/kfLnBhE+bfila3hbMp2roKj8kk0uKeASpfeXbnZgvIbr713fEUnTj9LjcTZ2WLJ0dvLE/xgoLNaoOQyx1wyPyh61Yur4Dkih

In [13]:
login_data={
"__VIEWSTATE":VIEWSTATE,
"__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
"__EVENTVALIDATION":EVENTVALIDATION,
"ddlShareholdingDay":'21',
"ddlShareholdingMonth":'06',
"ddlShareholdingYear":"2018",
'txtStockCode':'91888'
           }


In [14]:
r=s.post(URL, data=login_data)

In [16]:
print(r.url)

http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [35]:
print(driver.find_element_by_id("content").text)

Here is some important text you want to retrieve!
A button to click!


In [31]:
driver.close()

In [3]:
browser = webdriver.Chrome()

In [13]:
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [11]:
from selenium.webdriver.common.keys import Keys

In [15]:
element = browser.find_element_by_id("ddlShareholdingDay")

In [16]:
element.send_keys("15")

In [17]:
element

<selenium.webdriver.remote.webelement.WebElement (session="4927b9dee3910b452742e0e3db6445fd", element="0.857161784554014-1")>

In [18]:
element = browser.find_element_by_xpath("//select[@name='ddlShareholdingDay']")

In [20]:
all_options = element.find_elements_by_tag_name("option")

In [31]:
browser.find_element_by_id("btnSearch").click()

In [35]:
html_txt = browser.page_source

In [36]:
soup = BeautifulSoup(html_txt,'html.parser')

In [37]:
first_part = get_first_part()
print(first_part)

{'last_date': '2018-06-20', 'stock_code': '91888', 'stock_name': '中國國旅'}


In [40]:
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [43]:
browser.back()

In [83]:
browser = webdriver.Chrome()
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [44]:
begin_date = ''
end_date = ''

In [88]:
i[8:10]

'20'

In [92]:
temp5 = temp4[-10:-1]

In [93]:
temp5

10031    2018-06-06
10032    2018-06-07
10033    2018-06-08
10036    2018-06-11
10037    2018-06-12
10038    2018-06-13
10039    2018-06-14
10040    2018-06-15
10044    2018-06-19
Name: calendarDate, dtype: object

In [95]:
temp_time

'2018-06-06'

In [97]:
for temp_time in temp5:
    
    select = Select(browser.find_element_by_name('ddlShareholdingMonth'))
    select.select_by_value(temp_time[5:7])

    select = Select(browser.find_element_by_name('ddlShareholdingDay'))
    select.select_by_value(temp_time[8:10])

    select = Select(browser.find_element_by_name('ddlShareholdingYear'))
    select.select_by_value(temp_time[0:4])

    element = browser.find_element_by_id("txtStockCode")
    element.send_keys("91888")

    browser.find_element_by_id("btnSearch").click()

    html_txt = browser.page_source

    soup = BeautifulSoup(html_txt,'html.parser')
    
    print(get_first_part())

    browser.back()

{'last_date': '2018-06-06', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-07', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-08', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-11', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-12', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-13', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-14', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-15', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-19', 'stock_code': '91888', 'stock_name': '中國國旅'}


In [45]:
import tushare as ts

In [46]:
temp = ts.trade_cal()

In [104]:
begin_date = '2018-01-01'
end_date = '2018-06-20'

In [98]:
s1 = '20180612'

In [105]:
    trade_cal = temp
    trade_cal = trade_cal[trade_cal['calendarDate'] >= begin_date]
    trade_cal = trade_cal[trade_cal['calendarDate'] <= end_date]
    trade_cal = trade_cal[trade_cal['isOpen'] == 1]
    trade_cal = trade_cal['calendarDate']

In [106]:
trade_cal

9876     2018-01-02
9877     2018-01-03
9878     2018-01-04
9879     2018-01-05
9882     2018-01-08
9883     2018-01-09
9884     2018-01-10
9885     2018-01-11
9886     2018-01-12
9889     2018-01-15
9890     2018-01-16
9891     2018-01-17
9892     2018-01-18
9893     2018-01-19
9896     2018-01-22
9897     2018-01-23
9898     2018-01-24
9899     2018-01-25
9900     2018-01-26
9903     2018-01-29
9904     2018-01-30
9905     2018-01-31
9906     2018-02-01
9907     2018-02-02
9910     2018-02-05
9911     2018-02-06
9912     2018-02-07
9913     2018-02-08
9914     2018-02-09
9917     2018-02-12
            ...    
10003    2018-05-09
10004    2018-05-10
10005    2018-05-11
10008    2018-05-14
10009    2018-05-15
10010    2018-05-16
10011    2018-05-17
10012    2018-05-18
10015    2018-05-21
10016    2018-05-22
10017    2018-05-23
10018    2018-05-24
10019    2018-05-25
10022    2018-05-28
10023    2018-05-29
10024    2018-05-30
10025    2018-05-31
10026    2018-06-01
10029    2018-06-04


In [101]:
s1[6:8]

'12'

In [61]:
temp2 = temp1[temp1['calendarDate'] <= end_date]

In [63]:
temp3 = temp2[temp2['isOpen'] == 1]

In [67]:
temp4=temp3['calendarDate']

In [75]:
time1 = temp4.iat[0]

In [76]:
import datetime

In [77]:
time2 = datetime.datetime.strptime(time1,'%Y-%m-%d')

In [82]:
for i in temp4:
    print(i)

2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-02-03
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-05
2017-04-06
2017-04-07
2017-04-10
2017-04-11
2017-04-12
2017-04-13
2017-04-14
2017-04-17
2017-04-18
2017-04-19
2017-04-20
2017-04-21
2017-04-24
2017-04-25
2017-04-26
2017-04-27
2017-04-28
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-15
2017-05-16
2017-05-17
2017-05-18
2017-05-19